In [1]:
import sys
#sys.path.append('/content/drive/My Drive/DQN/input/')
sys.path.append('/home/kesci/input')
import math, random
import gym
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from common8945.wrappers import make_atari, wrap_deepmind, wrap_pytorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.transforms as T

In [17]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.show()
    
def plot2(frame_idx, rewards, losses, reward_episodes, loss_frames):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. average reward for every %s episode(s): %s' % (frame_idx, reward_episodes, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(132)
    plt.title('frame %s. average loss for every %s frame(s)' % (frame_idx, loss_frames))
    plt.plot(losses)
    plt.show()

In [2]:
#hyper-parameters
DEV = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPSILON_BEGIN = 0.99
EPSILON_END = 0.01
EPSILON_TOTAL = 1000000  
BATCH_SIZE = 32
NO_OP = 0
LEARNING_RATE = 0.0000625
GAMMA = 0.99
MEMORY_CAPACITY = 2 ** 15
ATOMS_NUM = 51
VMIN = -10.0
VMAX = 10.0
DELTA = 1.0 * (VMAX - VMIN) / (ATOMS_NUM - 1)
TARGET_UPDATE = 1000
EPISODE = 100000
LEARNING_START = MEMORY_CAPACITY + 10000
PRINT_START = MEMORY_CAPACITY + 20000
PROBABILITY_MIN = 0.01
PROBABILITY_MAX = 0.99
PRIORITIZATION_EPS = 0.0001
PRIORITIZATION_ALPHA = 0.6
PRIORITIZATION_BETA = 0.4
PRIORITIZATION_DELTA_BETA = 0.00000003

In [19]:
class SegmentTree(object):
  
  def __init__(self, capacity):
    self.capacity = capacity
    self.sum = [0.0 for i in range (capacity << 1)]
    self.min_ = [1.0 for i in range (capacity << 1)]

  def update(self, index, val):
    index += self.capacity
    self.sum[index] = val ** PRIORITIZATION_ALPHA
    self.min_[index] = val ** PRIORITIZATION_ALPHA
    while index != 1:
      index >>= 1
      self.sum[index] = self.sum[index << 1] + self.sum[index << 1 | 1]
      self.min_[index] = min(self.min_[index << 1], self.min_[index << 1 | 1])
  
  def query(self, val):
    rt = 1
    while rt < self.capacity:
      if val < self.sum[rt << 1]:
        rt = rt << 1
      else: 
        val -= self.sum[rt << 1]
        rt = rt << 1 | 1 
    return rt - self.capacity, self.sum[rt]
  
  def query_sum(self):
    return self.sum[1]

  def query_min(self):
    return self.min_[1]


class PrioritizedMemory(object):

    def __init__(self):
        self.tree = SegmentTree(MEMORY_CAPACITY)
        self.memory = []
        self.position = 0
        self.max_priority = 1.0

    def clamp_error(self, error):
        if error > 1:
          return 1
        if error < 0:
          return 0
        return error

    def push(self, state, action, reward, next_state, done):
        if len(self.memory) < MEMORY_CAPACITY:
            self.memory.append(None)
        self.tree.update(self.position, self.max_priority)
        self.memory[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % MEMORY_CAPACITY

    def sample(self, batch_size, beta):
        indexes, weights, states, actions, rewards, next_states, dones = [], [], [], [], [], [], []
        average = self.tree.query_sum() / batch_size
        min_priority = self.tree.query_min()
        sum_priority = self.tree.query_sum()
        min_priority = min_priority / sum_priority
        max_weight = (min_priority * MEMORY_CAPACITY) ** (-beta)
        for i in range(batch_size):
          l = average * i
          r = average * (i + 1)
          val = random.uniform(l, r) #[l, r)
          index, priority = self.tree.query(val)
          state, action, reward, next_state, done = self.memory[index]
          indexes.append(index)
          states.append(state)
          actions.append(action)
          rewards.append(reward)
          next_states.append(next_state)
          dones.append(done)
          priority = priority / sum_priority
          weight = (priority * MEMORY_CAPACITY) ** (-beta)
          weights.append(weight / max_weight)
        return indexes, weights, states, actions, rewards, next_states, dones

    def update_errors(self, indexes, priorities):
      for i in range(BATCH_SIZE):
        self.tree.update(indexes[i], priorities[i] + PRIORITIZATION_EPS)
        self.max_priority = max(self.max_priority, priorities[i])
        
    def size(self):
        return len(self.memory)

In [20]:
class FactorisedGaussianNoise(nn.Module):
  def __init__(self, input, output, init = 0.5):
    super(FactorisedGaussianNoise, self).__init__()
    self.input = input
    self.output = output
    self.mu_w = nn.Parameter(torch.FloatTensor(output, input))
    self.sigma_w = nn.Parameter(torch.FloatTensor(output, input))
    self.register_buffer('eps_w', torch.FloatTensor(output, input))
    self.mu_b = nn.Parameter(torch.FloatTensor(output))
    self.sigma_b = nn.Parameter(torch.FloatTensor(output))
    self.register_buffer('eps_b', torch.FloatTensor(output))
    
    #initialize parameters
    range = 1.0 / math.sqrt(input)
    self.mu_w.data.uniform_(-range, range)
    self.sigma_w.data.fill_(init / math.sqrt(input))
    self.mu_b.data.uniform_(-range, range)
    self.sigma_b.data.fill_(init / math.sqrt(output))
    self.reset()

  def forward(self, x):
    if self.training:
      return F.linear(x, self.mu_w + self.sigma_w * self.eps_w, self.mu_b + self.sigma_b * self.eps_b)
    else:
      return F.linear(x, self.mu_w, self.mu_b)
      
  def reset(self):
    eps_input = self.random_noise(self.input)
    eps_output = self.random_noise(self.output)
    self.eps_w.copy_(eps_output.ger(eps_input))
    self.eps_b.copy_(eps_output)

  def random_noise(self, size):
    res = torch.randn(size)
    return res.sign() * res.abs().sqrt()

In [21]:
class RainbowDQN(nn.Module):

    def __init__(self, input_size, atoms_num, action_space):
        super(RainbowDQN, self).__init__()
        
        self.atoms_num = atoms_num
        self.action_space = action_space
        
        self.conv1 = nn.Conv2d(input_size, 32, kernel_size = 8, stride = 4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 4, stride = 2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size = 3, stride = 1)
        
        self.fc_value = FactorisedGaussianNoise(7 * 7 * 64, 512)
        self.output_value = FactorisedGaussianNoise(512, atoms_num)
        
        self.fc_advantage = FactorisedGaussianNoise(7 * 7 * 64, 512)
        self.output_advantage = FactorisedGaussianNoise(512, atoms_num * action_space)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))

        x_value = F.relu(self.fc_value(x.view(x.size(0), -1)))
        x_value = self.output_value(x_value)
        x_value = x_value.view(x.size(0), 1, self.atoms_num)
        
        x_advantage = F.relu(self.fc_advantage(x.view(x.size(0), -1)))
        x_advantage = self.output_advantage(x_advantage)
        x_advantage = x_advantage.view(x.size(0), self.action_space, self.atoms_num)
        
        x = x_value + x_advantage - x_advantage.mean(1, keepdim = True)
        x = F.softmax(x.view(-1, self.atoms_num), dim = 1).view(-1, self.action_space, self.atoms_num)
        
        return x   
    
    def reset(self):
      self.fc_value.reset()
      self.output_value.reset()
      self.fc_advantage.reset()
      self.output_advantage.reset()

    def evaluate(self):
      self.eval()
      self.fc_value.eval()
      self.output_value.eval()
      self.fc_advantage.eval()
      self.output_advantage.eval()

In [22]:
class Agent: 
    def __init__(self, action_space, model_path = None):
        self.action_space = action_space
        self.memory = PrioritizedMemory()
        self.policy_net = RainbowDQN(1, ATOMS_NUM, self.action_space).to(DEV)
        self.target_net = RainbowDQN(1, ATOMS_NUM, self.action_space).to(DEV)
        if model_path == None:
            self.play_mode = False
        else:
            self.paly_mode = True
            self.policy_net = torch.load(model_path) #,  map_location=torch.device('cpu'))
        self.target_net.load_state_dict(self.policy_net.state_dict())
        #self.optimizer = optim.RMSprop(self.policy_net.parameters(), lr = EARNING_RATE, eps = 0.001, alpha = 0.95)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr = LEARNING_RATE)#, eps = 0.000015)
        
    def take_action(self, state):
        with torch.no_grad():
          output = self.policy_net(state)
          z = torch.linspace(VMIN, VMAX, ATOMS_NUM, dtype = torch.float, device = DEV)
          output = output * z
          return output.sum(2).max(1)[1]
    
    def act(self, frame):
        with torch.no_grad():
            state = torch.tensor(frame / 255.0, device = DEV, dtype = torch.float).unsqueeze(0) 
            output = self.policy_net(state)
            z = torch.linspace(VMIN, VMAX, ATOMS_NUM, dtype = torch.float, device = DEV)
            output = output * z
            return output.sum(2).max(1)[1]
   
    def optimize(self, frame_idx):
        beta = min(1.0, PRIORITIZATION_BETA + PRIORITIZATION_DELTA_BETA * frame_idx)
       # print(PRIORITIZATION_BETA, PRIORITIZATION_DELTA_BETA, frame_idx, beta)
        indexes, weights, states, actions, rewards, next_states, dones = self.memory.sample(BATCH_SIZE, beta)
        batch_states = torch.cat(states).to(DEV)
        batch_next_states = torch.cat(next_states).to(DEV)
        batch_weights = torch.tensor(weights, device = DEV, dtype = torch.float)
        batch_actions = torch.tensor(actions, device = DEV, dtype = torch.long)
        batch_rewards = torch.tensor(rewards, device = DEV, dtype = torch.float) 
        batch_dones = torch.tensor(dones, device = DEV, dtype = torch.float)
        
        policy_output = self.policy_net(batch_states)
        policy_action = batch_actions.unsqueeze(1).unsqueeze(1).expand(BATCH_SIZE, 1, ATOMS_NUM)
        policy_value = policy_output.gather(1, policy_action).squeeze(1)
        
        z = torch.linspace(VMIN, VMAX, ATOMS_NUM, dtype = torch.float, device = DEV)
        target_output = self.target_net(batch_next_states) * z
        target_action = target_output.sum(2).max(1)[1]
        target_action = target_action.unsqueeze(1).unsqueeze(1).expand(BATCH_SIZE, 1, ATOMS_NUM)
        target_value = target_output.gather(1, target_action).squeeze(1)
        
        batch_dones = batch_dones.unsqueeze(1).expand(BATCH_SIZE, ATOMS_NUM)
        batch_rewards = batch_rewards.unsqueeze(1).expand(BATCH_SIZE, ATOMS_NUM)
        z = z.unsqueeze(0).expand(BATCH_SIZE, ATOMS_NUM)
        Tz = batch_rewards + GAMMA * (1 - batch_dones) * z
        Tz.clamp_(VMIN, VMAX)
        b  = (Tz - VMIN) / DELTA
        l  = b.floor().long()
        u  = b.ceil().long()
        idx = torch.linspace(0, (BATCH_SIZE - 1) * ATOMS_NUM, BATCH_SIZE, dtype = torch.float, device = DEV).long()
        idx = idx.unsqueeze(1).expand(BATCH_SIZE, ATOMS_NUM)
        m = torch.zeros(BATCH_SIZE * ATOMS_NUM, dtype = torch.float, device = DEV)    
        m.index_add_(0, (l + idx).view(-1), (target_value * (u.float() - b)).view(-1))
        m.index_add_(0, (u + idx).view(-1), (target_value * (b - l.float())).view(-1))
        m = m.view(BATCH_SIZE, ATOMS_NUM)
        
        policy_value.data.clamp_min_(PROBABILITY_MIN)
        m.data.clamp_min_(PROBABILITY_MIN)
        loss = -(m * policy_value.log()).sum(1)
        td_errors = loss.clone().to(DEV)
        td_errors = td_errors.data.clamp_min_(0.0)
        self.memory.update_errors(indexes, td_errors.detach().cpu().numpy().tolist())

        loss = loss * batch_weights
        loss = loss.mean()

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        self.policy_net.reset()
        self.target_net.reset()

        return loss.item()
    
    def store_experience(self, state, action, reward, next_state, done):
        self.memory.push(state.to('cpu'), action.to('cpu'), reward, next_state.to('cpu'), done)

In [23]:
#env_id = "PongNoFrameskip-v4"
#train(env_id, '/home/kesci/work/0510/1/', 1, 1000)
#train(env_id, '/content/drive/My Drive/DQN/0510/1', 5, 5000)
# You may choose the environment from these environment setting below:
# Pong: PongNoFrameskip-v4
# Krull: KrullNoFrameskip-v4
# Tutankham: TutankhamNoFrameskip-v4
# Atlantis: AtlantisNoFrameskip-v4
# Freeway: FreewayNoFrameskip-v4
# Beam Rider: BeamRiderNoFrameskip-v4

In [ ]:
from itertools import count
directory = '/content/drive/My Drive/DQN/0515/8/'
    
env_id = "BeamRiderNoFrameskip-v4"
env    = make_atari(env_id)
env    = wrap_deepmind(env, clip_rewards = False)
env    = wrap_pytorch(env)
agent = Agent(env.action_space.n)
 
all_losses = []#torch.load('/content/drive/My Drive/DQN/0515/8/loss12700.pth')
all_rewards = []#torch.load('/content/drive/My Drive/DQN/0515/8/reward12700.pth')
frame_idx = 0
reward_episodes = 1
loss_frames = 1000
sum_reward = 0
sum_loss = 0
sum_reward = 0
frame_idx = 0

for episode in range(1, EPISODE):
    episode_reward = 0
    frame = env.reset()
    state = torch.tensor(frame / 255.0, device = DEV, dtype = torch.float).unsqueeze(0) 
    
    for t in count():
        frame_idx += 1
        agent.policy_net.reset()
        action = agent.take_action(state)
        next_frame, reward, done, _ = env.step(action)
        episode_reward += reward
        next_state = torch.tensor(next_frame / 255.0, device = DEV, dtype = torch.float).unsqueeze(0)
        agent.store_experience(state, action, reward, next_state, done)
        state = next_state
        
        loss = 0
        if frame_idx > LEARNING_START:
            loss = agent.optimize(frame_idx)
            if frame_idx % TARGET_UPDATE == 0:
                agent.target_net.load_state_dict(agent.policy_net.state_dict())
        
        if frame_idx > PRINT_START:
            sum_loss += loss
            if frame_idx % loss_frames == 0:
                sum_loss = sum_loss / loss_frames
                all_losses.append(sum_loss)
                sum_loss = 0
        
        if done:
            break
    
    sum_reward += episode_reward
    if episode % reward_episodes == 0:
        sum_reward /= reward_episodes
        all_rewards.append(sum_reward)
        plot2(frame_idx, all_rewards, all_losses, reward_episodes, loss_frames)
        sum_reward = 0
        
    if episode % 100 == 0 or episode_reward >= 10000:
        torch.save(agent.policy_net, directory + 'network' + str(episode) + '.pth')
        torch.save(all_rewards, directory + 'reward' + str(episode) + '.pth')
        torch.save(all_losses, directory + 'loss' + str(episode) + '.pth')
    
env.close()